# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    ##print(file_path_list)

/home/jlonzo/Lesson4_Project


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in master csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Casandra Data Structure creation, Data load and Code Testing 

## The event_datafile_new.csv Master CSV file contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [5]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
)
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Query 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [7]:
## Query 1.1:  Return artist, song title and song's length for music heard during sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS songplays_bySessionId "
query = query + "(artist text, song_title text, song_length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [8]:
## Query 1.2: Load Master CSV file (event_datafile_new.csv) into songplays_bySessionId Table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_bySessionId (artist, song_title, song_length, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3]) ))

In [9]:
## Query 1.3: Verify the data was loaded and table satisfies User Requirements
query = "SELECT * FROM songplays_bySessionId WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame()    
for row in rows:
    ##print (row.artist, row.song_title, row.song_length)
    df = df.append(pd.DataFrame(data={'artist':[row.artist], 'title':[row.song_title], ' length (sec.)':[row.song_length]}))

df

,artist,title,length (sec.)
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
## Query 2.1: Return artist, song title (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS songplays_byUserId "
query = query + "(artist text, song_title text, user_name text, userId int, sessionId int, itemInSession int, PRIMARY KEY ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [11]:
## Query 2.2: Load Master CSV file (event_datafile_new.csv) into songplays_byUserId Table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:      
        query = "INSERT INTO songplays_byUserId (artist, song_title, user_name, userId, sessionId, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], ' '.join(line[x] for x in (1,4)), int(line[10]), int(line[8]), int(line[3]) ))

In [25]:
## Query 2.3: Verify the data was loaded and table satisfies User Requirements
query = "SELECT * FROM songplays_byUserId WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df2 = pd.DataFrame()     
for row in rows:
    ##print (row.artist, row.song_title, row.user_name)
    df2 = df2.append(pd.DataFrame(data={'artist':[row.artist], 'title':[row.song_title], ' user name':[row.user_name]}))

df2

,artist,title,user name
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
0,Three Drives,Greece 2000,Sylvie Cruz
0,Sebastien Tellier,Kilometer,Sylvie Cruz
0,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


### Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [27]:
## Query 3.1: Return every user name (first and last) who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS userName_bySongTitle "
query = query + "(user_name text, song_title text, userId int, PRIMARY KEY (song_title,userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [28]:
## Query 3.2: Load Master CSV file (event_datafile_new.csv) into userName_bySongTitle Table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:      
        query = "INSERT INTO userName_bySongTitle (user_name, song_title, userId)"
        query = query + "VALUES (%s, %s, %s)"
        session.execute(query, (' '.join(line[x] for x in (1,4)), line[9], int(line[10]) ))

In [29]:
## Query 3.3: Verify the data was loaded and table satisfies User Requirements
query = "SELECT * FROM userName_bySongTitle WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df3 = pd.DataFrame()      
for row in rows:
    ##print (row.user_name)
    df3 = df3.append(pd.DataFrame(data={'user name':[row.user_name]}))

df3

,user name
0,Jacqueline Lynch
0,Tegan Levine
0,Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
query = "DROP TABLE IF EXISTS songplays_bySessionId"
rows = session.execute(query)

In [ ]:
query = "DROP TABLE IF EXISTS songplays_byUserId"
rows = session.execute(query)

In [ ]:
query = "DROP TABLE IF EXISTS userName_bySongTitle"
rows = session.execute(query)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()